# `xWRF` overview

`xWRF` is a package designed to make the post-processing of `WRF` output data more pythonic. It's aim is to smooth the rough edges around the unique, non CF-compliant `WRF` output data format and make the data accessible to utilities like [`dask`](https://dask.org/) and the wider [Pangeo](https://pangeo.io/) universe.

It is built as an [Accessor](https://xarray.pydata.org/en/stable/internals/extending-xarray.html) on top of [`xarray`](https://xarray.pydata.org/en/stable/index.html), providing a very simple user interface.

## Examining the data

When opening up a normal `WRF` output file with the simple `xarray` netcdf backend, one can see that it does not provide a lot of useful information.

In [1]:
import xwrf

ds_old = xwrf.tutorial.open_dataset("wrfout")
ds_old

<xarray.Dataset>
Dimensions:  (south_north: 185, west_east: 200, bottom_top: 41,
              west_east_stag: 201, south_north_stag: 186)
Coordinates:
    XLAT     (south_north, west_east) float32 ...
    XLONG    (south_north, west_east) float32 ...
    XTIME    datetime64[ns] ...
Dimensions without coordinates: south_north, west_east, bottom_top,
                                west_east_stag, south_north_stag
Data variables:
    ZNU      (bottom_top) float32 ...
    Times    |S19 ...
    U        (bottom_top, south_north, west_east_stag) float32 ...
    V        (bottom_top, south_north_stag, west_east) float32 ...
    T        (bottom_top, south_north, west_east) float32 ...
    QVAPOR   (bottom_top, south_north, west_east) float32 ...
    P        (bottom_top, south_north, west_east) float32 ...
    PB       (bottom_top, south_north, west_east) float32 ...
    PSN      (south_north, west_east) float32 ...
Attributes: (12/149)
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2018-10-15_00:00:00
    SIMULATION_START_DATE:           2018-10-15_00:00:00
    WEST-EAST_GRID_DIMENSION:        201
    SOUTH-NORTH_GRID_DIMENSION:      186
    BOTTOM-TOP_GRID_DIMENSION:       42
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

While all variables are present, e.g. the information about the projection is still in the metadata and also for some fields, there are non-`metpy` compliant units attributes.

So let's try to use the standard `xWRF.postprocess()` function.

In [2]:
ds_new = xwrf.tutorial.open_dataset("wrfout").xwrf.postprocess()
ds_new

<xarray.Dataset>
Dimensions:                    (y: 185, x: 200, z: 41, x_stag: 201, y_stag: 186)
Coordinates:
    XLAT                       (y, x) float32 ...
    XLONG                      (y, x) float32 ...
    XTIME                      datetime64[ns] ...
  * x                          (x) float64 -8.794e+04 -8.694e+04 ... 1.111e+05
  * y                          (y) float64 -9.949e+04 -9.849e+04 ... 8.451e+04
  * y_stag                     (y_stag) float64 -9.999e+04 ... 8.501e+04
  * x_stag                     (x_stag) float64 -8.844e+04 ... 1.116e+05
  * z                          (z) float32 0.9969 0.9899 ... 0.008311 0.002679
    Time                       datetime64[ns] 2018-10-16
Data variables:
    Times                      |S19 b'2018-10-16_00:00:00'
    U                          (z, y, x_stag) float32 ...
    V                          (z, y_stag, x) float32 ...
    QVAPOR                     (z, y, x) float32 ...
    PSN                        (y, x) float32 ...
    wrf_projection             object +proj=lcc +x_0=0 +y_0=0 +a=6370000 +b=6...
    air_potential_temperature  (z, y, x) float32 293.1 293.0 ... 496.7 496.7
    air_pressure               (z, y, x) float32 9.606e+04 ... 5.255e+03
Attributes: (12/149)
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2018-10-15_00:00:00
    SIMULATION_START_DATE:           2018-10-15_00:00:00
    WEST-EAST_GRID_DIMENSION:        201
    SOUTH-NORTH_GRID_DIMENSION:      186
    BOTTOM-TOP_GRID_DIMENSION:       42
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

As you see, `xWRF` added some coordinate data, reassigned some dimensions and generally increased the amount of information available in the dataset.

## Projection treatment

As you can see, `xWRF` has determined the correct projection from the netCDF metadata and has added a `wrf_projection` variable to the dataset storing the [`pyproj`](https://pyproj4.github.io/pyproj/stable/) projection object.

In [3]:
ds_new['wrf_projection'].item()

<Derived Projected CRS: +proj=lcc +x_0=0 +y_0=0 +a=6370000 +b=6370000 +lat ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Lambert Conic Conformal (2SP)
Datum: unknown
- Ellipsoid: unknown
- Prime Meridian: Greenwich

Using this projection `xWRF` has also calculated the regular model grid, which the WRF simulation is performed on and which can be used for e.g. bilinear interpolation.

In [4]:
ds_new[['x', 'y']]

<xarray.Dataset>
Dimensions:  (x: 200, y: 185)
Coordinates:
  * x        (x) float64 -8.794e+04 -8.694e+04 ... 1.101e+05 1.111e+05
  * y        (y) float64 -9.949e+04 -9.849e+04 ... 8.351e+04 8.451e+04
    XLAT     (y, x) float32 48.66 48.66 48.66 48.66 ... 50.31 50.31 50.31 50.31
    XLONG    (y, x) float32 7.302 7.316 7.329 7.343 ... 10.02 10.04 10.05 10.07
    XTIME    datetime64[ns] 2018-10-16
    Time     datetime64[ns] 2018-10-16
Data variables:
    *empty*
Attributes: (12/149)
    TITLE:                            OUTPUT FROM WRF V4.2.1 MODEL
    START_DATE:                      2018-10-15_00:00:00
    SIMULATION_START_DATE:           2018-10-15_00:00:00
    WEST-EAST_GRID_DIMENSION:        201
    SOUTH-NORTH_GRID_DIMENSION:      186
    BOTTOM-TOP_GRID_DIMENSION:       42
    ...                              ...
    ISLAKE:                          21
    ISICE:                           15
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      2
    ETAC:                            0.2

## Attribute changes

`xWRF` adds additional attributes to variables in order to make them [CF](https://cfconventions.org/)- and [COMODO](https://web.archive.org/web/20160417032300/http://pycomodo.forge.imag.fr/norm.html)-compliant. It also amends `unit` attributes to work with [`metpy`](https://unidata.github.io/MetPy/latest/index.html) units, enabling a seamless integration with the [Pangeo](https://pangeo.io/) software stack.

Here, for example the x-wind component gets the correct [CF](https://cfconventions.org/) `standard_name` and a [COMODO](https://web.archive.org/web/20160417032300/http://pycomodo.forge.imag.fr/norm.html) `grid_mapping` attribute indicating the respective projection.

In [5]:
ds_old['U'].attrs, ds_new['U'].attrs

({'FieldType': 104,
  'MemoryOrder': 'XYZ',
  'description': 'x-wind component',
  'units': 'm s-1',
  'stagger': 'X'},
 {'FieldType': 104,
  'MemoryOrder': 'XYZ',
  'description': 'x-wind component',
  'units': 'm s-1',
  'stagger': 'X',
  'standard_name': 'x_wind',
  'grid_mapping': 'wrf_projection'})

Also, the `units` attribute of the `PSN` variable was cleaned up to conform to [`metpy`](https://unidata.github.io/MetPy/latest/index.html) unit conventions.

*Note: As of now, unit translations are implemented on a manual basis, so please raise an [issue](https://github.com/ncar-xdev/xwrf/issues/new?assignees=&labels=bug%2Ctriage&template=bugreport.yml&title=%5BBug%5D%3A+) with us if you encounter any problems in this regard. In the future, this will be implemented in a more structured manner.*

In [6]:
ds_old['PSN'].attrs['units'], ds_new['PSN'].attrs['units']

('umol co2/m2/s', 'umol/m2/s')

In [7]:
import metpy

try:
    ds_old['PSN'].metpy.quantify()
except metpy.units.UndefinedUnitError as e:
    print(e)
ds_new['PSN'].metpy.quantify()

'co' is not defined in the unit registry


Magnitude,[[0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] ... [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0] [0.0 0.0 0.0 ... 0.0 0.0 0.0]]
Units,micromole/(meter2 second)


## Diagnostic variables

Because some WRF output fields are quite raw, essential diagnostic variables like the air pressure or air potential temperature are missing. These get added by `xWRF` by default. Users can choose to keep the fields after the computation of diagnostics is done by using `.xwrf.postprocess(drop_diagnostic_variable_components=False)`.

In [8]:
ds_new['air_pressure']

<xarray.DataArray 'air_pressure' (z: 41, y: 185, x: 200)>
array([[[96061.33  , 96289.305 , 96542.164 , ..., 94000.52  ,
         94060.18  , 94125.93  ],
        [96354.19  , 96559.54  , 96788.15  , ..., 93986.94  ,
         94053.16  , 94131.41  ],
        [96618.875 , 96802.72  , 97009.68  , ..., 93942.86  ,
         94024.41  , 94109.89  ],
        ...,
        [97941.016 , 98106.45  , 98263.31  , ..., 97029.516 ,
         97210.54  , 97364.65  ],
        [97972.73  , 98145.05  , 98303.4   , ..., 96860.61  ,
         97058.305 , 97225.63  ],
        [98005.805 , 98186.09  , 98350.09  , ..., 96682.766 ,
         96889.836 , 97071.96  ]],

       [[95421.78  , 95644.06  , 95897.99  , ..., 93370.6   ,
         93408.16  , 93476.91  ],
        [95710.63  , 95912.01  , 96138.766 , ..., 93360.945 ,
         93423.914 , 93484.27  ],
        [95971.42  , 96153.22  , 96358.664 , ..., 93315.125 ,
         93395.42  , 93477.78  ],
...
        [ 5789.4277,  5789.6885,  5789.6763, ...,  5789.2373,
          5789.17  ,  5789.231 ],
        [ 5789.4307,  5789.68  ,  5789.678 , ...,  5789.197 ,
          5789.115 ,  5789.226 ],
        [ 5789.425 ,  5789.432 ,  5789.4414, ...,  5789.23  ,
          5789.233 ,  5789.2236]],

       [[ 5254.4307,  5254.4272,  5254.4346, ...,  5254.779 ,
          5254.791 ,  5254.7827],
        [ 5254.433 ,  5254.5454,  5254.5435, ...,  5254.584 ,
          5254.583 ,  5254.7754],
        [ 5254.415 ,  5254.531 ,  5254.541 , ...,  5254.5635,
          5254.576 ,  5254.763 ],
        ...,
        [ 5254.223 ,  5254.474 ,  5254.489 , ...,  5254.607 ,
          5254.6274,  5255.0107],
        [ 5254.2217,  5254.4785,  5254.4897, ...,  5254.6113,
          5254.641 ,  5255.0127],
        [ 5254.2266,  5254.227 ,  5254.2363, ...,  5255.0146,
          5255.005 ,  5255.012 ]]], dtype=float32)
Coordinates:
    XLAT     (y, x) float32 48.66 48.66 48.66 48.66 ... 50.31 50.31 50.31 50.31
    XLONG    (y, x) float32 7.302 7.316 7.329 7.343 ... 10.02 10.04 10.05 10.07
    XTIME    datetime64[ns] 2018-10-16
  * x        (x) float64 -8.794e+04 -8.694e+04 ... 1.101e+05 1.111e+05
  * y        (y) float64 -9.949e+04 -9.849e+04 ... 8.351e+04 8.451e+04
  * z        (z) float32 0.9969 0.9899 0.981 ... 0.01452 0.008311 0.002679
    Time     datetime64[ns] 2018-10-16
Attributes:
    units:          Pa
    standard_name:  air_pressure